In [1]:
#!pip install --upgrade azureml-automl-core

In [2]:
#!pip install --upgrade azureml-sdk[automl]

In [3]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
import os
import shutil
import joblib

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import ScriptRunConfig
from azureml.core import Environment

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


## Workspace initialization and experiment creation

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment=Experiment(workspace = ws, name ="hd-experiment")



print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136277
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-136277


## Dataset

In [8]:
# loading dataset 
dataset = Dataset.get_by_name(ws, 'Diabetes-dataset')

#df = pd.read_csv('https://raw.githubusercontent.com/eaamankwah/Deploy-Diabetes-model-with-AzureML/main/diabetes.csv')



## Config Compute Cluster

In [9]:
cpu_cluster = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Setting environment for hyperdrive run

In [10]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment.get(workspace=ws, name="AzureML-AutoML")

In [11]:
with open('score.py') as f:
    print(f.read())

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"Pregnancies": pd.Series([0], dtype="int64"), "Glucose": pd.Series([0], dtype="int64"), "BloodPressure": pd.Series([0], dtype="int64"), "SkinThickness": pd.Series([0], dtype="int64"), "Insulin": pd.Series([0], dtype="int64"), "BMI": pd.Series([0.0], dtype="float64"), "DiabetesPedigreeFunct

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [23]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
        "--C": choice(0.2,0,5,1.0),
        "--max_iter": choice(10, 20, 30)
    })

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = cpu_cluster,
                      environment=env)

hyperdrive_config = HyperDriveConfig(
                                hyperparameter_sampling = ps,
                                policy = policy,
                                primary_metric_name = 'Accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 20,
                                max_concurrent_runs = 4,
                                run_config = src)

In [24]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef
Web View: https://ml.azure.com/experiments/hd-experiment/runs/HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-136277/workspaces/quick-starts-ws-136277

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-27T18:30:13.173744][API][INFO]Experiment created<END>\n""<START>[2021-01-27T18:30:13.784453][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-27T18:30:14.5837388Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-27T18:30:13.976113][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef
Web View: https://ml.azure.com/experiments/hd-experiment/runs/HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef?wsid=/subscriptions/b968fb36-

{'runId': 'HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-27T18:30:12.961724Z',
 'endTimeUtc': '2021-01-27T18:40:15.934998Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '42059655-7ee8-410f-a82e-966c0e0fee11',
  'score': '0.7272727272727273',
  'best_child_run_id': 'HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef_2',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg136277.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_a31bbc1a-ed2c-4e75-8218-3d15d44672e

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [26]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)
print('\n Accuracy: ', best_run_metrics['Accuracy'])

Best Run ID:  HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef_2

 Metrics:  {'Regularization Strength:': 0.2, 'Max iterations:': 30, 'Accuracy': 0.7272727272727273}

 Parameters:  ['--C', '0.2', '--max_iter', '30']

 Accuracy:  0.7272727272727273


In [27]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hd-experiment,HD_a31bbc1a-ed2c-4e75-8218-3d15d44672ef_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [29]:
#TODO: Save the best model

In [30]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service